The manual: http://data.atoc.org/sites/all/themes/atoc/files/RSPS5045.pdf

The data: http://data.atoc.org/fares-data

In [1]:
import pandas as pd

### Flow file

In [34]:
out = pd.read_table("../data/RJFAF575/RJFAF575.FFL")

In [28]:
#out_truncated = 
out[5:].head()

,/!! Start of file
5,RF0027003201000000AS3112299902012020ATO01Y0000002
6,RF0027003301000000AS3112299902012020ATO01Y0000034
7,RF0027003401000000AS3112299902012020ATO01Y0000121
8,RF0027003501000000AS3112299902012020ATO01Y0000174
9,RF0027006401000000AS3112299902012020ATO01Y0000049


In [26]:
flow_data = out.copy()

In [39]:
flow_data.rename(columns={"/!! Start of file                                                               ":"records"}, inplace=True)

In [50]:
flow_data["update_marker"] = flow_data["records"].str[0:1]
flow_data["record_type"] = flow_data["records"].str[1:2]
flow_data["origin_code"] = flow_data["records"].str[2:6]
flow_data["destination_code"] = flow_data["records"].str[6:10]
flow_data["route_code"] = flow_data["records"].str[10:15]
flow_data["status_code"] = flow_data["records"].str[15:18]
flow_data["usage_code"] = flow_data["records"].str[18:19]
flow_data["direction"] = flow_data["records"].str[19:20]
flow_data["end_date"] = flow_data["records"].str[20:28]
flow_data["start_date"] = flow_data["records"].str[28:36]
flow_data["toc"] = flow_data["records"].str[36:39]
flow_data["cross_london_ind"] = flow_data["records"].str[39:40]

In [51]:
flow_data

,records,update_marker,record_type,origin_code,destination_code,route_code,status_code,usage_code,direction,end_date,start_date,toc,cross_london_ind
0,/!! Content type: FFL ...,/,!,! Co,nten,t typ,e:,,F,FL,,,
1,/!! Sequence: 575 ...,/,!,! Se,quen,ce:,,,5,75,,,
2,/!! Records: 5938764 ...,/,!,! Re,cord,s:,,,5,938764,,,
3,/!! Generated: 18/05/2020 ...,/,!,! Ge,nera,ted:,,,1,8/05/202,0,,
4,/!! Exporter: RJIS ...,/,!,! Ex,port,er:,,,R,JIS,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5938765,RT07364537DS00005080,R,T,0736,4537,DS000,050,8,0,,,,
5938766,RT0736453CDR00000940B1,R,T,0736,453C,DR000,009,4,0,B1,,,
5938767,RT0736453SDR00001130,R,T,0736,453S,DR000,011,3,0,,,,
5938768,RT0736453SDS00000850,R,T,0736,453S,DS000,008,5,0,,,,


Locations file Locations, including group locations

In [10]:
locations_file = pd.read_table("../data/RJFAF575/RJFAF575.LOC")

In [20]:
locations_file.values[5]

array(['RG7000320200920191901201918012019LONDON ZONES 1-2     '],
      dtype=object)

In [23]:
locations_file.values[6]

array(['RG7000320311229992109201918012019LONDON ZONES 1-2     '],
      dtype=object)

In [24]:
locations_file.values[7]

array(['RG7000330200920191901201918012019LONDON ZONES 1-3     '],
      dtype=object)

### Data Feed